In [ ]:
# Import necessary libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from matplotlib.ticker import MaxNLocator

In [ ]:
# Change fonts and specify font size
plt.rcParams['font.family'] = 'serif'
plt.rcParams['mathtext.fontset'] = 'dejavuserif'
FONT_SIZE = 12

In [ ]:
# Define necessary functions
# Function to save files
def save_file(values, file_name, file_directory):
    
    # Save the file as a .npy file
    np.save(os.path.join(file_directory, file_name), values)
    
    print(f'Saved {file_name} to {file_directory}')

In [ ]:
# Residual Neural Network class
class RESIDUAL():

    def __init__(self, input_shape, output_shape):
        
        self.input_shape = input_shape
        self.output_shape = output_shape
        
        # Initialize input_layer here
        self.input_layer = None  

    # Method to build the hidden layers
    def build_hidden_layers(self):
        
        # Convolutional Layers
        # First Convolutional Layer
        x1 = tf.keras.layers.Conv1D(filters=200, kernel_size=32, padding='same', activation='LeakyReLU', kernel_initializer = 'glorot_normal')(self.input_layer)
        x1 = tf.keras.layers.BatchNormalization()(x1)

        # Second Convolutional Layer
        x2 = tf.keras.layers.Conv1D(filters=100, kernel_size=32, padding='same', activation='LeakyReLU', kernel_initializer = 'glorot_normal')(x1)
        x2 = tf.keras.layers.BatchNormalization()(x2)

        # Skip concatenated connection
        x2 = tf.keras.layers.Concatenate(axis=-1)([x1, x2])
        
        # Third Convolutional Layer
        x3 = tf.keras.layers.Conv1D(filters=400, kernel_size=32, padding='same', activation='LeakyReLU', kernel_initializer = 'glorot_normal')(x2)
        x3 = tf.keras.layers.BatchNormalization()(x3)

        # Skip concatenated connection
        x3 = tf.keras.layers.Concatenate(axis=-1)([x2, x3])

        return x3

    # Method to build the overall model
    def build_model(self):
        
        # Input layer
        self.input_layer = tf.keras.layers.Input(shape=self.input_shape)

        # Hidden layer
        hidden_layer = self.build_hidden_layers()

        # Output Layer
        output_layer = tf.keras.layers.Conv1D(filters=self.output_shape[1], kernel_size=32, padding='same', activation='LeakyReLU', kernel_initializer = 'glorot_normal')(hidden_layer)

        # Build model
        self.model = tf.keras.models.Model(inputs=[self.input_layer], outputs=[output_layer])

        return self.model

    # Method to compile the model
    def compile(self, optimizer, loss, evaluation_metric):
        
        # Compile model
        self.model.compile(optimizer=optimizer, loss=loss, metrics=evaluation_metric)

        return self.model
    
    # Define method to train the model
    def train(self, x_train, y_train, epochs, batch_size, callbacks):
        
        # Train model
        self.history = self.model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, callbacks=callbacks, validation_split=0.2)
        
        return self.history
    
    # Method to print summary of model
    def summary(self):
        
        self.model.summary()

In [ ]:
class plots:
    
    def __init__(self, history, file_directory):

        self.history = history
        self.file_directory = file_directory

    def loss(self):

        loss_name = list(self.history.history.keys())[0]

        # Training
        loss = self.history.history[loss_name]
        val_loss = self.history.history['val_' + loss_name]

        loss_plot = plt.figure()
        epochs = range(1, len(loss)+1)
        plt.plot(epochs, loss, 'bo--', label = 'Training Loss', markersize = 2)
        plt.plot(epochs, val_loss, 'go--', label = 'Validation Loss', markersize = 2)
        plt.title('Training and Validation Loss', fontsize=FONT_SIZE)
        plt.xlabel('Epochs', fontsize=FONT_SIZE)
        plt.ylabel('Loss', fontsize=FONT_SIZE)
        plt.legend(['Training Loss', 'Validation Loss'], fontsize=FONT_SIZE)
        ax = loss_plot.gca()
        ax.xaxis.set_major_locator(MaxNLocator(integer=True))
        plt.savefig(self.file_directory + '/loss.pdf', bbox_inches='tight')
        
        return loss_plot

    def evaluation_metric(self):

        metric_name = list(self.history.history.keys())[1]
        
        # Training
        metric = self.history.history[metric_name]
        val_metric = self.history.history['val_' + metric_name]

        metric_plot = plt.figure()
        epochs = range(1, len(metric)+1)
        plt.plot(epochs, metric, 'bo--', label = 'Training Metric', markersize = 2)
        plt.plot(epochs, val_metric, 'go--', label = 'Validation Metric', markersize = 2)
        plt.title('Training and Validation Evaluation Metric', fontsize=FONT_SIZE)
        plt.xlabel('Epochs', fontsize=FONT_SIZE)
        plt.ylabel('Evaluation Metric', fontsize=FONT_SIZE)
        plt.legend(['Training Metric', 'Validation Metric'], fontsize=FONT_SIZE)
        ax = metric_plot.gca()
        ax.xaxis.set_major_locator(MaxNLocator(integer=True))
        plt.savefig(self.file_directory + '/evaluation_metric.pdf', bbox_inches='tight')

        return metric_plot

In [ ]:
# Define directories
current_directory = os.getcwd()

# Define directory for the normalized data
normalized_data_directory = os.path.join(current_directory, '..', '..', 'data', 'normalized')

# Define directory for the trained results
trained_results_directory = os.path.join(current_directory, '..', '..', 'residual', 'training_results')

In [ ]:
# Working with just the displacement data to conserve memory
# Load the normalized training subsets for displacement data
print('Loading the normalized training subsets for displacement data...')
normalized_training_displacement_data = np.load(os.path.join(normalized_data_directory, 'normalized_training_displacement_data.npy'))

In [ ]:
# Load the normalized training subsets for force data
print('Loading the normalized training subsets for force data...')
normalized_training_force_data = np.load(os.path.join(normalized_data_directory, 'normalized_training_force_data.npy'))

In [ ]:
# Print the shapes of the displacement and force data
print(f'The shape of displacement data is {normalized_training_displacement_data.shape[1:]}.')
print(f'The shape of force data is {normalized_training_force_data.shape[1:]}.')

In [ ]:
# Define variables that remain constant during the training
input_shape = normalized_training_displacement_data.shape[1:]
output_shape = normalized_training_force_data.shape[1:]

In [ ]:
# Create an instance of the Residual class
model = RESIDUAL(input_shape, output_shape)

In [ ]:
# Build and the model
model.build_model()
model.compile(optimizer = tf.keras.optimizers.Nadam(learning_rate=0.001), loss = 'mae', evaluation_metric = 'mse')

In [ ]:
# Print the model summary
model.summary()

In [ ]:
# Define callbacks
# Early stopping callback
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 10, mode = 'min', restore_best_weights = True)

In [ ]:
# Train the model
print('Training the model...')
history = model.train(normalized_training_displacement_data, normalized_training_force_data, epochs = 500, batch_size = 32, callbacks=[early_stopping_callback])

In [ ]:
plot = plots(history, trained_results_directory)
loss_plot = plot.loss()
evaluation_metric_plot = plot.evaluation_metric()

In [ ]:
# Save the model
print('Saving the model...')
model.model.save(os.path.join(trained_results_directory, 'model.h5'))